<a href="https://colab.research.google.com/github/dlkt101101/AMATH445/blob/main/AMATH445_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMATH 445
## Assignment 2
Prepared by: Darren Alexander Lam Kin Teng (20977843)

# Question 1

In [65]:
import torch
import numpy as np
import pandas as pd
from torch.nn.modules.activation import ReLU
from torch.nn import Tanh
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import matplotlib as mp
import seaborn as sns
import matplotlib.pyplot as plt

In [66]:
data = pd.read_csv('/content/Q1_bioprinting_data.csv')

In [67]:
data.shape

(617, 51)

In [68]:
data.head()

,Reference,DOI,Final_Alginate_Conc_(%w/v),Final_Gelatin_Conc_(%w/v),Final_GelMA_Conc_(%w/v),Final_Hyaluronic_Acid_Conc_(%w/v),Final_MeHA_Conc_(%w/v),Final_NorHA_Conc_(%w/v),Final_Fibroin/Fibrinogen_Conc_(%w/v),Final_P127_Conc_(%w/v),...,Saline_Solution_Used?,EtOH_Solution_Used?,Photoinitiator_Used?,Enzymatic_Crosslinker_Used?,Matrigel_Used?,Conical_or_Straight_Nozzle,Primary/Not_Primary,Viability_at_time_of_observation_(%),Acceptable_Viability_(Yes/No),Acceptable_Pressure_(Yes/No)
0,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
1,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,72.0,N,N
2,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,80.0,Y,Y
3,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
4,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,77.0,N,N


In [69]:
df = data.copy()
df.drop(labels=['Reference', 'DOI'], axis =1, inplace=True)

In [70]:
df.isna().sum()

,0
Final_Alginate_Conc_(%w/v),0
Final_Gelatin_Conc_(%w/v),0
Final_GelMA_Conc_(%w/v),0
Final_Hyaluronic_Acid_Conc_(%w/v),0
Final_MeHA_Conc_(%w/v),0
Final_NorHA_Conc_(%w/v),0
Final_Fibroin/Fibrinogen_Conc_(%w/v),0
Final_P127_Conc_(%w/v),0
Final_Collagen_Conc_(%w/v),1
Final_Chitosan_Conc_(%w/v),0


## Data Preprocessing

Imputting mode temperatures

In [71]:
df['Fiber_Diameter_(µm)'].unique()

array([ 430.    , 1280.    ,       nan, 1052.1   , 1103.09  ,  917.23  ,
       1132.66  , 1015.34  , 1560.65  ,  909.19  ,  858.15  ,  833.96  ,
        370.2   ,  336.92  ,  287.01  ,  308.09  ,  328.74  ,  286.84  ,
        298.04  ,  296.7   ,  443.96  ,  624.48  ,   68.27  ,  611.93  ,
        595.83  ,  774.06  ,  666.599 ,  708.89  ,  910.8   ,  342.708 ,
        333.28  ,  362.06  ,  348.62  ,  871.1   ,  336.12  ,  366.04  ,
        903.52  ,  223.36  ,  486.18  ,  238.61  ,  519.54  , 1001.65  ,
        498.2   ,  466.5   ,  466.6   ,  319.21  ,  334.34  ,  527.77  ,
        517.29  ,  625.71  ,  630.01  ,  713.    ,  620.95  ,  682.35  ,
        592.02  ,  462.65  ,  460.74  ,  542.53  ,  636.51  ,  608.85  ,
        648.04  ,  294.71  ,  419.    ,  281.83  ,  409.    , 1553.07  ,
       1687.146 , 1675.869 , 1300.498 , 1233.966 , 1417.279 , 1014.45  ,
       1101.27  , 1311.52  , 1373.11  , 1302.86  , 1496.41  ,  292.39  ,
        357.32  ,  291.28  ,  297.81  ,  964.53  , 

In [72]:
imputer_mode = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
df.loc[:,['Syringe_Temperature_(°C)','Substrate_Temperature_(°C)']] = imputer_mode.fit_transform(
    df.loc[:,['Syringe_Temperature_(°C)','Substrate_Temperature_(°C)']])

In [73]:
df.columns

Index(['Final_Alginate_Conc_(%w/v)', 'Final_Gelatin_Conc_(%w/v)',
       'Final_GelMA_Conc_(%w/v)', 'Final_Hyaluronic_Acid_Conc_(%w/v)',
       'Final_MeHA_Conc_(%w/v)', 'Final_NorHA_Conc_(%w/v)',
       'Final_Fibroin/Fibrinogen_Conc_(%w/v)', 'Final_P127_Conc_(%w/v)',
       'Final_Collagen_Conc_(%w/v)', 'Final_Chitosan_Conc_(%w/v)',
       'Final_CS-AEMA_Conc_(%w/v)', 'Final_TCP_Conc_(%w/v)',
       'Final_Gellan_Conc_(%w/v)', 'Final_Nano/Methycellulose_Conc_(%w/v)',
       'Final_PEGTA_Conc_(%w/v)', 'Final_PEGMA_Conc_(%w/v)',
       'Final_PEGDA_Conc_(%w/v)', 'Final_Agarose_Conc_(%w/v)',
       'CaCl2_Conc_(mM)', 'NaCl2_Conc_(mM)', 'BaCl2_Conc_(mM)',
       'SrCl2_Conc_(mM)', 'Physical_Crosslinking_Durantion_(s)',
       'Photocrosslinking_Duration_(s)', 'Extrusion_Pressure (kPa)',
       'Extrusion_Rate_Lengthwise_(mm/s)', 'Extrusion_Rate_Volume-wise_(mL/s)',
       'Nozzle_Movement_Speed_(mm/s)', 'Inner_Nozzle_Outer_Diameter_(µm)',
       'Outer_Nozzle_Inner_Diameter_(µm)', 'Fiber

Dropping variables and instances

In [74]:
# Drop certain material concentration as no concentration values exist in papers# Variables where more than 50% of all instances have null values are dropped
# This amounts to variables with 309 or more null instances for the cell viability dataset with 617 instances
df = df.dropna(axis = 1, thresh=309)

# Variables where more than 50% of all instances have null values are dropped
# This amounts to variables with 177 or more null instances
# Extrusion Pressure with 354 instances
# df = df.dropna(axis = 1, thresh=177)

# Drop instances without cell viability values
df = df[df['Viability_at_time_of_observation_(%)'].notna()]

# Drop nonprinting instances (instances were extrusion pressure is zero)
df = df.drop(df[df['Extrusion_Pressure (kPa)'] == 0 ].index)
df = df[df['Extrusion_Pressure (kPa)'].notna()]

# used to create extrusion pressure dataset
df.head()

,Final_Alginate_Conc_(%w/v),Final_Gelatin_Conc_(%w/v),Final_GelMA_Conc_(%w/v),Final_Hyaluronic_Acid_Conc_(%w/v),Final_MeHA_Conc_(%w/v),Final_NorHA_Conc_(%w/v),Final_Fibroin/Fibrinogen_Conc_(%w/v),Final_P127_Conc_(%w/v),Final_Collagen_Conc_(%w/v),Final_Chitosan_Conc_(%w/v),...,Saline_Solution_Used?,EtOH_Solution_Used?,Photoinitiator_Used?,Enzymatic_Crosslinker_Used?,Matrigel_Used?,Conical_or_Straight_Nozzle,Primary/Not_Primary,Viability_at_time_of_observation_(%),Acceptable_Viability_(Yes/No),Acceptable_Pressure_(Yes/No)
0,2.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
1,2.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,...,N,N,N,N,N,S,Primary,72.0,N,N
2,2.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,...,N,N,N,N,N,S,Primary,80.0,Y,Y
3,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
4,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0,...,N,N,N,N,N,S,Primary,77.0,N,N


In [75]:
# Create the independent variables (x) set and the dependent variable (y) set from the training dataset. The cell
# viability or other variable name for y need to be change to the full variable name in the dataset used.
# Drop for cell viability, extrusion pressure, and Intrastudy dataset creation

df = df.drop(['Final_PEGTA_Conc_(%w/v)','Final_PEGMA_Conc_(%w/v)'], axis = 1)


Imputting values

In [76]:
df['Fiber_Diameter_(µm)'].unique()

array([ 430.   , 1280.   ,      nan, 1052.1  , 1103.09 ,  917.23 ,
       1132.66 , 1015.34 , 1560.65 ,  909.19 ,  858.15 ,  833.96 ,
        370.2  ,  336.92 ,  287.01 ,  308.09 ,  328.74 ,  286.84 ,
        298.04 ,  296.7  ,  443.96 ,  774.06 ,  666.599,  708.89 ,
        910.8  ,  342.708,  333.28 ,  362.06 ,  348.62 ,  871.1  ,
        336.12 ,  366.04 ,  903.52 ,  223.36 ,  486.18 ,  238.61 ,
        519.54 ,  319.21 ,  334.34 ,  527.77 ,  517.29 ,  625.71 ,
        630.01 ,  713.   ,  620.95 ,  682.35 ,  592.02 ,  462.65 ,
        460.74 ,  542.53 ,  294.71 ,  419.   ,  281.83 ,  409.   ,
       1401.869, 1561.572, 1233.18 , 1251.31 , 1015.78 , 1074.07 ,
        568.017,  543.08 ,  488.83 ,  574.24 ,  643.59 ,  651.22 ,
        793.49 ,  716.9  ,  462.   , 1000.   , 1174.605, 1185.569,
       1974.411, 1347.79 , 1430.34 , 1215.505, 1319.245, 1385.098,
       1210.715, 1240.192, 1059.551, 1274.433, 1315.53 , 1190.   ,
       1103.19 , 1227.6  , 1136.08 ,  819.92 ,  700.35 ,  677.

In [77]:
imputer_knn = KNNImputer(n_neighbors = 30, weights = "uniform") #imputing mode value into missing values
df.iloc[:,0:22] = imputer_knn.fit_transform(df.iloc[:,0:22]) #used for cell viability dataset preprocessing

In [78]:
df['Fiber_Diameter_(µm)'].unique()

array([ 430.   , 1280.   ,      nan, 1052.1  , 1103.09 ,  917.23 ,
       1132.66 , 1015.34 , 1560.65 ,  909.19 ,  858.15 ,  833.96 ,
        370.2  ,  336.92 ,  287.01 ,  308.09 ,  328.74 ,  286.84 ,
        298.04 ,  296.7  ,  443.96 ,  774.06 ,  666.599,  708.89 ,
        910.8  ,  342.708,  333.28 ,  362.06 ,  348.62 ,  871.1  ,
        336.12 ,  366.04 ,  903.52 ,  223.36 ,  486.18 ,  238.61 ,
        519.54 ,  319.21 ,  334.34 ,  527.77 ,  517.29 ,  625.71 ,
        630.01 ,  713.   ,  620.95 ,  682.35 ,  592.02 ,  462.65 ,
        460.74 ,  542.53 ,  294.71 ,  419.   ,  281.83 ,  409.   ,
       1401.869, 1561.572, 1233.18 , 1251.31 , 1015.78 , 1074.07 ,
        568.017,  543.08 ,  488.83 ,  574.24 ,  643.59 ,  651.22 ,
        793.49 ,  716.9  ,  462.   , 1000.   , 1174.605, 1185.569,
       1974.411, 1347.79 , 1430.34 , 1215.505, 1319.245, 1385.098,
       1210.715, 1240.192, 1059.551, 1274.433, 1315.53 , 1190.   ,
       1103.19 , 1227.6  , 1136.08 ,  819.92 ,  700.35 ,  677.

In [79]:
df.iloc[:,:22].columns

Index(['Final_Alginate_Conc_(%w/v)', 'Final_Gelatin_Conc_(%w/v)',
       'Final_GelMA_Conc_(%w/v)', 'Final_Hyaluronic_Acid_Conc_(%w/v)',
       'Final_MeHA_Conc_(%w/v)', 'Final_NorHA_Conc_(%w/v)',
       'Final_Fibroin/Fibrinogen_Conc_(%w/v)', 'Final_P127_Conc_(%w/v)',
       'Final_Collagen_Conc_(%w/v)', 'Final_Chitosan_Conc_(%w/v)',
       'Final_CS-AEMA_Conc_(%w/v)', 'Final_TCP_Conc_(%w/v)',
       'Final_Gellan_Conc_(%w/v)', 'Final_Nano/Methycellulose_Conc_(%w/v)',
       'Final_PEGDA_Conc_(%w/v)', 'Final_Agarose_Conc_(%w/v)',
       'CaCl2_Conc_(mM)', 'NaCl2_Conc_(mM)', 'BaCl2_Conc_(mM)',
       'SrCl2_Conc_(mM)', 'Physical_Crosslinking_Durantion_(s)',
       'Photocrosslinking_Duration_(s)'],
      dtype='object')

Imputting categorical values

In [80]:
df['Fiber_Diameter_(µm)'].unique()

array([ 430.   , 1280.   ,      nan, 1052.1  , 1103.09 ,  917.23 ,
       1132.66 , 1015.34 , 1560.65 ,  909.19 ,  858.15 ,  833.96 ,
        370.2  ,  336.92 ,  287.01 ,  308.09 ,  328.74 ,  286.84 ,
        298.04 ,  296.7  ,  443.96 ,  774.06 ,  666.599,  708.89 ,
        910.8  ,  342.708,  333.28 ,  362.06 ,  348.62 ,  871.1  ,
        336.12 ,  366.04 ,  903.52 ,  223.36 ,  486.18 ,  238.61 ,
        519.54 ,  319.21 ,  334.34 ,  527.77 ,  517.29 ,  625.71 ,
        630.01 ,  713.   ,  620.95 ,  682.35 ,  592.02 ,  462.65 ,
        460.74 ,  542.53 ,  294.71 ,  419.   ,  281.83 ,  409.   ,
       1401.869, 1561.572, 1233.18 , 1251.31 , 1015.78 , 1074.07 ,
        568.017,  543.08 ,  488.83 ,  574.24 ,  643.59 ,  651.22 ,
        793.49 ,  716.9  ,  462.   , 1000.   , 1174.605, 1185.569,
       1974.411, 1347.79 , 1430.34 , 1215.505, 1319.245, 1385.098,
       1210.715, 1240.192, 1059.551, 1274.433, 1315.53 , 1190.   ,
       1103.19 , 1227.6  , 1136.08 ,  819.92 ,  700.35 ,  677.

In [81]:
df['Conical_or_Straight_Nozzle'].value_counts().index[0]

'S'

In [82]:
df = df.fillna(df['Conical_or_Straight_Nozzle'].value_counts().index[0])

In [83]:
df['Fiber_Diameter_(µm)'].unique()

array([430.0, 1280.0, 'S', 1052.1, 1103.09, 917.23, 1132.66, 1015.34,
       1560.65, 909.19, 858.15, 833.96, 370.2, 336.92, 287.01, 308.09,
       328.74, 286.84, 298.04, 296.7, 443.96, 774.06, 666.599, 708.89,
       910.8, 342.708, 333.28, 362.06, 348.62, 871.1, 336.12, 366.04,
       903.52, 223.36, 486.18, 238.61, 519.54, 319.21, 334.34, 527.77,
       517.29, 625.71, 630.01, 713.0, 620.95, 682.35, 592.02, 462.65,
       460.74, 542.53, 294.71, 419.0, 281.83, 409.0, 1401.869, 1561.572,
       1233.18, 1251.31, 1015.78, 1074.07, 568.017, 543.08, 488.83,
       574.24, 643.59, 651.22, 793.49, 716.9, 462.0, 1000.0, 1174.605,
       1185.569, 1974.411, 1347.79, 1430.34, 1215.505, 1319.245, 1385.098,
       1210.715, 1240.192, 1059.551, 1274.433, 1315.53, 1190.0, 1103.19,
       1227.6, 1136.08, 819.92, 700.35, 677.57, 355.88, 608.54, 763.25,
       350.0, 370.0, 310.0, 973.0, 300.0, 1014.53, 1113.08, 715.2, 709.13,
       690.94, 573.26, 400.0, 293.09, 372.59, 441.59, 524.5], dtype=ob

Normalizing and scaling

In [84]:
df['Fiber_Diameter_(µm)'].unique()

array([430.0, 1280.0, 'S', 1052.1, 1103.09, 917.23, 1132.66, 1015.34,
       1560.65, 909.19, 858.15, 833.96, 370.2, 336.92, 287.01, 308.09,
       328.74, 286.84, 298.04, 296.7, 443.96, 774.06, 666.599, 708.89,
       910.8, 342.708, 333.28, 362.06, 348.62, 871.1, 336.12, 366.04,
       903.52, 223.36, 486.18, 238.61, 519.54, 319.21, 334.34, 527.77,
       517.29, 625.71, 630.01, 713.0, 620.95, 682.35, 592.02, 462.65,
       460.74, 542.53, 294.71, 419.0, 281.83, 409.0, 1401.869, 1561.572,
       1233.18, 1251.31, 1015.78, 1074.07, 568.017, 543.08, 488.83,
       574.24, 643.59, 651.22, 793.49, 716.9, 462.0, 1000.0, 1174.605,
       1185.569, 1974.411, 1347.79, 1430.34, 1215.505, 1319.245, 1385.098,
       1210.715, 1240.192, 1059.551, 1274.433, 1315.53, 1190.0, 1103.19,
       1227.6, 1136.08, 819.92, 700.35, 677.57, 355.88, 608.54, 763.25,
       350.0, 370.0, 310.0, 973.0, 300.0, 1014.53, 1113.08, 715.2, 709.13,
       690.94, 573.26, 400.0, 293.09, 372.59, 441.59, 524.5], dtype=ob

In [85]:
X = df.drop(labels=['Viability_at_time_of_observation_(%)'], axis=1)
y = df['Viability_at_time_of_observation_(%)']

# Xclass = df.drop(labels=['Acceptable_Viability_(Yes/No)'], axis=1)
# yclass = df['Acceptable_Viability_(Yes/No)']

In [86]:
df['Fiber_Diameter_(µm)'].unique()

array([430.0, 1280.0, 'S', 1052.1, 1103.09, 917.23, 1132.66, 1015.34,
       1560.65, 909.19, 858.15, 833.96, 370.2, 336.92, 287.01, 308.09,
       328.74, 286.84, 298.04, 296.7, 443.96, 774.06, 666.599, 708.89,
       910.8, 342.708, 333.28, 362.06, 348.62, 871.1, 336.12, 366.04,
       903.52, 223.36, 486.18, 238.61, 519.54, 319.21, 334.34, 527.77,
       517.29, 625.71, 630.01, 713.0, 620.95, 682.35, 592.02, 462.65,
       460.74, 542.53, 294.71, 419.0, 281.83, 409.0, 1401.869, 1561.572,
       1233.18, 1251.31, 1015.78, 1074.07, 568.017, 543.08, 488.83,
       574.24, 643.59, 651.22, 793.49, 716.9, 462.0, 1000.0, 1174.605,
       1185.569, 1974.411, 1347.79, 1430.34, 1215.505, 1319.245, 1385.098,
       1210.715, 1240.192, 1059.551, 1274.433, 1315.53, 1190.0, 1103.19,
       1227.6, 1136.08, 819.92, 700.35, 677.57, 355.88, 608.54, 763.25,
       350.0, 370.0, 310.0, 973.0, 300.0, 1014.53, 1113.08, 715.2, 709.13,
       690.94, 573.26, 400.0, 293.09, 372.59, 441.59, 524.5], dtype=ob

In [87]:
X.iloc[:, :28].dtypes

,0
Final_Alginate_Conc_(%w/v),float64
Final_Gelatin_Conc_(%w/v),float64
Final_GelMA_Conc_(%w/v),float64
Final_Hyaluronic_Acid_Conc_(%w/v),float64
Final_MeHA_Conc_(%w/v),int64
Final_NorHA_Conc_(%w/v),int64
Final_Fibroin/Fibrinogen_Conc_(%w/v),float64
Final_P127_Conc_(%w/v),int64
Final_Collagen_Conc_(%w/v),float64
Final_Chitosan_Conc_(%w/v),int64


In [88]:
df['Fiber_Diameter_(µm)'].unique()

array([430.0, 1280.0, 'S', 1052.1, 1103.09, 917.23, 1132.66, 1015.34,
       1560.65, 909.19, 858.15, 833.96, 370.2, 336.92, 287.01, 308.09,
       328.74, 286.84, 298.04, 296.7, 443.96, 774.06, 666.599, 708.89,
       910.8, 342.708, 333.28, 362.06, 348.62, 871.1, 336.12, 366.04,
       903.52, 223.36, 486.18, 238.61, 519.54, 319.21, 334.34, 527.77,
       517.29, 625.71, 630.01, 713.0, 620.95, 682.35, 592.02, 462.65,
       460.74, 542.53, 294.71, 419.0, 281.83, 409.0, 1401.869, 1561.572,
       1233.18, 1251.31, 1015.78, 1074.07, 568.017, 543.08, 488.83,
       574.24, 643.59, 651.22, 793.49, 716.9, 462.0, 1000.0, 1174.605,
       1185.569, 1974.411, 1347.79, 1430.34, 1215.505, 1319.245, 1385.098,
       1210.715, 1240.192, 1059.551, 1274.433, 1315.53, 1190.0, 1103.19,
       1227.6, 1136.08, 819.92, 700.35, 677.57, 355.88, 608.54, 763.25,
       350.0, 370.0, 310.0, 973.0, 300.0, 1014.53, 1113.08, 715.2, 709.13,
       690.94, 573.26, 400.0, 293.09, 372.59, 441.59, 524.5], dtype=ob

In [89]:
X.loc[:,['Fiber_Diameter_(µm)', 'Cell_Density_(cells/mL)']]

,Fiber_Diameter_(µm),Cell_Density_(cells/mL)
0,430.0,250000.0
1,430.0,250000.0
2,430.0,250000.0
3,430.0,250000.0
4,1280.0,250000.0
...,...,...
612,S,10000000.0
613,S,10000000.0
614,524.5,10000000.0
615,S,10000000.0


In [90]:
# X.iloc[:,0:28] = MinMaxScaler().fit_transform(X.iloc[:,0:28]) # Used for cell viability generalized dataset

One Hot Encoding

In [91]:
X_ohencoded = pd.get_dummies(X, columns=['Cell_Culture_Medium_Used?',
                                         'DI_Water_Used?',
                                         'Precrosslinking_Solution_Used?',
                                         'Saline_Solution_Used?',
                                         'EtOH_Solution_Used?',
                                         'Photoinitiator_Used?',
                                         'Enzymatic_Crosslinker_Used?',
                                         'Matrigel_Used?',
                                         'Conical_or_Straight_Nozzle',
                                         'Primary/Not_Primary']) #one-hot encoding is used to encode binary/categorical data in datasets

y_ohencoded = pd.get_dummies(y)
y_ohencoded.isna().sum()

,0
31.00,0
33.00,0
34.60,0
36.00,0
40.00,0
...,...
98.33,0
98.86,0
99.00,0
99.50,0


## Model Development

In [92]:
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.15)